In [6]:
from openai import OpenAI
import json
import ast
from pathlib import Path
import traceback
from tqdm.auto import tqdm
import httpx
import time

llm_url="http://localhost:8000/v1/"
model="Qwen/Qwen2.5-3B-Instruct"

def wait_for_vllm_ready(timeout=300):
    print("⏳ Waiting for LLM API to be ready...")
    start_time = time.time()
    while True:
        try:
            response = httpx.get(llm_url+"models", timeout=2)
            if response.status_code == 200:
                print("✅ LLM is ready!")
                break
        except Exception as e:
            if time.time() - start_time > timeout:
                raise TimeoutError("❌ Timed out waiting for LLM.") from e
            time.sleep(5)

wait_for_vllm_ready()

# initial LLM
client = OpenAI(
    base_url='http://localhost:8000/v1',
    api_key='EMPTY', 
)

⏳ Waiting for LLM API to be ready...
✅ LLM is ready!


In [9]:
from ipywidgets import widgets
conversation = [dict(role="system", content="You are a helpful assistant")]
in_text = widgets.Text()
in_text.continuous_update = False
output = widgets.Output()

def text_eventhandler(*args):
    # Needed bc when we "reset" the text input
    # it fires instantly another event since
    # we "changed" it's value to ""
    if args[0]["new"] == "":
        return

    question = args[0]["new"]
    with output:
        print(f"User: {question}")

    conversation.append(dict(role="user",content=question))

    # Reset text field
    args[0]["owner"].value = ""
    response = client.chat.completions.create(messages=conversation, model=model).choices[0].message.content
    with output:
        print(f"AI: {response}")
    conversation.append(dict(role="system", content=response))
in_text.observe(text_eventhandler, "value")
widgets.VBox([in_text, output])